# `SELECT`

Nesta aula vamos explorar mais sobre o comando `SELECT`, usado para consultar a base de dados. 

Instale a base de teste 'musica' usando o script `musica.sql`. Este script instala a base de dados de exemplo do livro-texto da disciplina, de modo que vocês podem praticar com os exercícios do livro também!

Eis aqui o diagrama do modelo relacional dessa base de dados:

<center><img src="imgs/modelo_relacional.png" width=800/></center>

Eis a lista completa de chaves primárias e estrangeiras:

```SQL
ALTER TABLE AUTOR
    ADD PRIMARY KEY (Codigo_Autor);

ALTER TABLE CD
    ADD PRIMARY KEY (Codigo_CD) ;

ALTER TABLE GRAVADORA
    ADD PRIMARY KEY (Codigo_Gravadora) ;

ALTER TABLE FAIXA
    ADD PRIMARY KEY (Codigo_Musica, Codigo_CD);

ALTER TABLE MUSICA
    ADD PRIMARY KEY (Codigo_Musica) ;

ALTER TABLE MUSICA_AUTOR
    ADD PRIMARY KEY (Codigo_Musica, Codigo_Autor);

ALTER TABLE CD
    ADD FOREIGN KEY (Codigo_Gravadora)
        REFERENCES GRAVADORA(Codigo_Gravadora);

ALTER TABLE CD
    ADD FOREIGN KEY (CD_Indicado)
        REFERENCES CD(Codigo_CD);

ALTER TABLE FAIXA
    ADD FOREIGN KEY (Codigo_CD)
        REFERENCES CD(Codigo_CD);

ALTER TABLE FAIXA
    ADD FOREIGN KEY (Codigo_Musica)
        REFERENCES MUSICA(Codigo_Musica);

ALTER TABLE MUSICA_AUTOR
    ADD FOREIGN KEY (Codigo_Autor)
        REFERENCES AUTOR(Codigo_Autor);

ALTER TABLE MUSICA_AUTOR
    ADD FOREIGN KEY (Codigo_Musica)
        REFERENCES MUSICA(Codigo_Musica);
```

**Atividade:** Quais destas tabelas são tabelas de relacionamento?

Vamos agora criar a conexão com o banco de dados e o objeto auxiliar de conexão. Aqui resolvi usar uma estratégia diferente de implementação que dá o mesmo resultado, para deixar de exemplo para vocês:

In [3]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='Megadados!',
    database='musica',
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

## Explorando a estrutura da base de dados

Agora podemos usar nosso helper para enviar comandos à base de dados. Vamos ver quais tabelas existem na base 'musica':

In [4]:
db('SHOW TABLES')

Executando query:
('AUTOR',)
('CD',)
('CD_CATEGORIA',)
('FAIXA',)
('GRAVADORA',)
('MUSICA',)
('MUSICA_AUTOR',)


Para saber qual o schema da tabela 'cd', podemos usar o comando '`DESCRIBE`'

In [5]:
db('DESCRIBE CD')

Executando query:
('Codigo_CD', b'int', 'NO', 'PRI', None, '')
('Codigo_Gravadora', b'int', 'YES', 'MUL', None, '')
('Nome_CD', b'varchar(60)', 'YES', '', None, '')
('Preco_Venda', b'int', 'YES', '', None, '')
('Data_Lancamento', b'date', 'YES', '', None, '')
('CD_Indicado', b'int', 'YES', 'MUL', None, '')


## Consultando a base de dados

Relembrando: vamos usar o comando '`SELECT`' para listar os conteudos da tabela 'cd'

In [6]:
db('SELECT * FROM CD')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


O comando acima lista todos os registros da tabela 'cd', com todas as colunas presentes:

![Seleção da tabela inteira](imgs/tudo.png)

Vamos agora selecionar apenas algumas colunas para exibir.

In [7]:
db('SELECT Nome_CD, Data_Lancamento FROM CD')

Executando query:
('Mais do Mesmo', datetime.date(1998, 10, 1))
('Bate-Boca', datetime.date(1999, 7, 1))
('Elis Regina - Essa Mulher', datetime.date(1989, 5, 1))
('A Força que nunca Seca', datetime.date(1998, 12, 1))
('Perfil', datetime.date(2001, 5, 1))
('Barry Manilow Greatest Hits Vol I', datetime.date(1991, 11, 1))
('Listen Without Prejudice', datetime.date(1991, 10, 1))


Agora vemos apenas as colunas escolhidas. Relembrando: a operação de seleção de colunas chama-se **projeção**.

![Projeção](imgs/projecao.png)

Vamos agora atuar na escolha de linhas, selecionando quais desejamos. Para escolher todas as linhas cujo Nome_CD terminem em 'a', podemos executar a query a seguir:

In [8]:
db("SELECT * FROM CD WHERE Nome_CD LIKE '%a'")

Executando query:
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)


Vemos apenas as linhas escolhidas. Recordando: a operação de filtragem de linhas apropriadas chama-se **seleção**.

![Seleção](imgs/selecao.png)

## Cláusula `WHERE`

A cláusula `WHERE` permite filtrar as linhas da tabela (ou tabelas - mais sobre *JOIN* daqui a pouco). Basta especificar a condição de filtragem: o resultado da query será o conjunto de linhas para as quais a condição é verdadeira.

Já vimos alguns usos da cláusula `WHERE` acima. Vamos ver mais exemplos:

In [9]:
# Queries equivalentes.
db('SELECT * FROM CD')
db('SELECT * FROM CD WHERE True')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)
Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


Qual o cd mais antigo que custa 13 reais ou mais?

In [10]:
db('''
SELECT 
    Nome_CD 
FROM 
    CD
WHERE 
    Preco_venda >= 13 
ORDER BY 
    Data_Lancamento ASC 
LIMIT 1
''')

Executando query:
('Elis Regina - Essa Mulher',)


### Vamos praticar queries!

Preencha os códigos a seguir com buscas adequadas. Consulte seu livro texto para descobrir como montar essas queries.

Qual o nome da música mais longa?

Quais gravadoras não tem endereço declarado?

Quais cds foram lançados na década de 90 e custam 10 reais ou menos?

In [ ]:
db("""
SELECT
    Nome_CD
FROM
    CD
WHERE
    YEAR(Data_Lancamento) >= 1990
    AND YEAR(Data_Lancamento) <= 1999
    AND Preco_Venda <= 10
""")

In [ ]:
db("""
SELECT
    Nome_CD
FROM
    CD
WHERE
    YEAR(Data_Lancamento) BETWEEN 1990 AND 1999
    AND Preco_Venda <= 10
""")

Quais cds receberam alguma recomendação? Liste seus ids.

## Agregação

Quantas músicas tem na base?

In [ ]:
db('SELECT COUNT(*) FROM MUSICA')

Note o comando `COUNT` acima. Trata-se de um comando de *agregação*: uma operação que trabalha em cima de todo o resultado da query e retorna um valor agregado para esses resultados. 

Usamos o argumento "\*" para indicar que queremos apenas contar quantas linhas não são nulas. Normalmente é melhor especificar direito a coluna sobre a qual queremos operar.

Por exemplo: qual a duração da musica mais longa?

In [ ]:
db('SELECT MAX(duracao) FROM MUSICA')

**Atividade:** Quantas musicas foram escritas pelo autor número 1 (Renato Russo)?

**Atividade:** Se eu fosse comprar uma cópia de cada CD da base, quanto gastaria?

## Variáveis

Verificando manualmente a tabela `musica` podemos encontrar o nome da musica mais longa:

In [ ]:
db('SELECT * from MUSICA')

Nenhuma surpresa aqui: é "Faroeste Caboclo"...

Agora suponha que eu quero localizar o *nome* da música mais comprida. Note que o comando a seguir NÃO FAZ SENTIDO (porque?):

```SQL
SELECT Nome_musica, MAX(duracao) FROM MUSICA;
```

Talvez se especificássemos que a duracao da música tem que coincidir com a duracao máxima...

In [ ]:
try:
    db('SELECT Nome_musica FROM MUSICA WHERE duracao=MAX(duracao)')
except mysql.connector.DatabaseError as e:
    print(f'DatabaseError: {e}')

Também não funciona: não podemos usar uma função de agrupamento dentro da cláusula `WHERE`. Como resolver isso então?

Primeiro descubra qual é a maior duração e guarde em uma *variável* ***no banco de dados***.

In [ ]:
db('SELECT MAX(duracao) INTO @max_duracao from MUSICA')

Podemos checar o valor dessa variavel agora:

In [ ]:
db('SELECT @max_duracao')

Agora podemos terminar o serviço!

In [ ]:
db('SELECT Nome_musica FROM MUSICA WHERE duracao=@max_duracao')

**Atividade**: Encontre todas as musicas com duração acima da média.

## Pesquisa em múltiplas tabelas

### Produto cartesiano

Quando listamos várias tabelas no comando `SELECT` temos como resultado o produto cartesiano destas. 

(Para os matemáticos: não se trata estritamente do produto cartesiano, pois não geramos um conjunto de duplas de tuplas. Geramos um "aplainamento" - *flattening* - do resultado, para cada linha.)

Por exemplo, se temos uma tabela com 2 linhas e outra com 3 linhas, o resultado terá 6 linhas, como ilustrado a seguir:

```SQL
DROP DATABASE IF EXISTS test;
CREATE DATABASE test;
USE test;

CREATE TABLE tab1 (
    coluna11 INT,
    coluna21 INT
);

CREATE TABLE tab2 (
    coluna12 INT,
    coluna22 INT
);

INSERT INTO tab1 VALUES (1, 2), (3, 4);
INSERT INTO tab2 VALUES (1000, 2000), (3000, 4000), (5000, 6000);

SELECT * FROM tab1, tab2;
```

Resultado:

![Produto cartesiano](imgs/cartesiano.png)

Observe o que acontece quando unimos as tabelas *cd* e *gravadora*

In [ ]:
db('SELECT * FROM CD')

In [ ]:
db('SELECT * FROM GRAVADORA')

In [ ]:
db('DESCRIBE CD')

In [ ]:
db('DESCRIBE GRAVADORA')

In [ ]:
db('SELECT * FROM CD, GRAVADORA')

In [ ]:
db('SELECT COUNT(*) FROM CD, GRAVADORA')

In [ ]:
db('SELECT COUNT(*) FROM CD')

In [ ]:
db('SELECT COUNT(*) FROM GRAVADORA')

### Inner join

E se filtrássemos o resultado anterior, mantendo apenas as linhas em que o código de gravadora na tabela *'cd'* coincida com o código de gravadora na tabela *'gravadora'*?

In [ ]:
db('''
SELECT 
    * 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

In [ ]:
db('''
SELECT 
    Nome_CD, Nome_Gravadora 
FROM
    CD as c, GRAVADORA as g 
WHERE
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

Opa, parece que com isso conseguimos associar a gravadora ao cd!

Estamos mantendo apenas as linhas do produto cartesiano onde o codigo da gravadora declarado na tabela cd bate com o codigo da gravadora declarado na tabela gravadora. Logo, nestas linhas, a informação proveniente da tabela cd e a informação proveniente da tabela gravadora estarão se referindo à mesma gravadora. Conseguimos, portanto, conectar as informações de cada CD com as informações da gravadora à qual aquele CD pertence.

Denominamos *"inner join"*, *"equi-join"* ou "união regular" a essa operação de junção de tabelas através da união de chave primária de uma com chave estrangeira de outra. Eu chamo informalmente de "vamos bater chaves!"

Vamos construir uma lista de cds e suas respectivas gravadoras:

In [ ]:
db('''
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

Vamos aprimorar nossa query para procurar apenas os CDs gravados pela 'SOM LIVRE':

In [ ]:
db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, 
    GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
    AND g.Nome_Gravadora = 'SOM LIVRE'
""")

Existem "apelidos" para a união regular, que produzem o mesmo resultado:

In [ ]:
db('''
SELECT 
    Nome_CD, 
    Nome_Gravadora 
FROM 
    CD 
    INNER JOIN GRAVADORA USING (Codigo_Gravadora)
''')

In [ ]:
db('''
SELECT
    Nome_CD,
    Nome_Gravadora
FROM
    CD
    INNER JOIN GRAVADORA ON CD.Codigo_Gravadora = GRAVADORA.Codigo_Gravadora
''')

**Atividade**: Construa a lista das musicas do Renato Russo.

### Self join

E se quisermos listar os nomes de CDs e seus respectivos CDs indicados? Veja a solução abaixo. Note o uso de dois *aliases* diferentes para a tabela cd.

In [ ]:
# Observe também o uso da função CONCAT(), só para ficar mais bonito!
db('''
SELECT
    CONCAT('"', c1.Nome_CD, '" indicado por "', c2.Nome_CD, '"')
FROM
    CD c1,
    CD c2
WHERE
    c1.Codigo_CD = c2.CD_Indicado
''')

### *'Non-equi-join'*

Chamamos de *'non-equi-join'* a uma junção de tabelas que não usa a igualdade de colunas como critério de junção. 

Podemos fazer muitas investigações úteis com junção de tabelas. Vamos acompanhar um exemplo (também está no livro-texto).

Na base 'musica' temos uma tabela estranha, de poucas linhas, chamada cd_categoria.

In [ ]:
db('DESCRIBE CD_CATEGORIA')

In [ ]:
db('SELECT * from CD_CATEGORIA')

Esta tabela lista as categorias de preço de CDs. Por exemplo, se um CD custa 9 reais, ele está na categoria 1.

Gostariamos de usar essa informação em conjunto com a tabela de CDs para obter a lista de CDs em cada categoria. Por onde começar?

Vamos montar o produto cartesiano desta tabela cd_categoria com a tabela de CDs e ver se algum *insight* aparece:

In [ ]:
db('SELECT * FROM CD, CD_CATEGORIA LIMIT 10')

Interessante! Como se trata do produto cartesiano, cada CD é listado repetidas vezes, uma para cada categoria de CDs. 

*Cada linha tem* ***o preço do CD*** *e um par de valores com* ***limites de preço de faixa***. Leia esta sentença quantas vezes for necessário.

E se filtrássemos esse resultado para manter apenas as linhas onde o preço do CD está dentro dos limites de preço?

In [ ]:
db("""
SELECT 
    * 
FROM 
    CD, 
    CD_CATEGORIA cat 
WHERE 
    CD.Preco_Venda BETWEEN cat.Menor_Preco AND cat.Maior_Preco
""")

Surgiu um pequeno problema: tem CD alocado para duas categorias ao mesmo tempo! Culpa do operador `BETWEEN` e da nossa interpretação da tabela `cd_categoria`. Vamos remover o `BETWEEN` e usar os limites do jeito que a gente quer:

In [ ]:
db("""
SELECT 
    * 
FROM 
    CD, 
    CD_CATEGORIA cat 
WHERE 
    CD.Preco_Venda >= cat.Menor_Preco 
    AND CD.Preco_Venda < cat.Maior_Preco
""")

Finalmente, vamos ajustar nossa query para buscar os nomes de CDs na faixa 2 de preço:

In [ ]:
db("""
SELECT 
    Nome_CD 
FROM 
    CD, 
    CD_CATEGORIA cat 
WHERE 
    CD.Preco_Venda >= cat.Menor_Preco 
    AND CD.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_categoria = 2
""")

#### Vamos praticar

Liste o nome do CD e de sua respectiva gravadora para CDs na categoria 2.

In [ ]:
db ("""
SELECT
    Nome_CD, Nome_Gravadora
FROM
    CD,
    CD_CATEGORIA cat,
    GRAVADORA g
WHERE
    CD.Codigo_Gravadora = g.Codigo_Gravadora
    AND CD.Preco_Venda >= cat.Maior_Preco
    AND CD.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_Categoria = 2
""")

Vamos agora fechar a conexão com este banco de dados, porque no próximo tópico vamos abrir uma conexão com outro banco de dados.

In [ ]:
connection.close()

### Outer join

Considere a seguinte base de testes: cada perigo tem um nome e cada comida tem um nome e um perigo associado. (Use o script `tranqueira.sql` para testar essa base.)

```SQL
DROP DATABASE IF EXISTS tranqueira;
CREATE DATABASE tranqueira;
USE tranqueira;

CREATE TABLE comida (
	id INT NOT NULL AUTO_INCREMENT,
	Nome VARCHAR(30),
    idPerigo INT,
    PRIMARY KEY (id)
);

CREATE TABLE perigo (
	id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(20),
    PRIMARY KEY (id)
);

ALTER TABLE comida ADD FOREIGN KEY (idPerigo) REFERENCES perigo (id);

INSERT INTO perigo VALUES (1, 'Cardiaco'), (2, 'Intestinal'), (3, 'Dermatologico'), (4, 'Mental');
INSERT INTO comida VALUES (1, 'Torresmo', 1), (2, 'Alface', NULL), (3, 'Coxinha', 2), (4, 'Espetinho', 2);

SELECT * FROM comida;
SELECT * FROM perigo;
```

![Tabela comida](imgs/comida.png)

![Tabela perigo](imgs/perigo.png)

In [ ]:
connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='megadados',
    database='tranqueira',
)

db = partial(run_db_query, connection)

Fazendo o inner join da tabela comida com a tabela perigo temos a lista de comidas que estão associadas a algum perigo:

In [ ]:
db('USE tranqueira')
db('''
SELECT 
    comida.Nome, perigo.Nome 
FROM 
    comida INNER JOIN perigo ON comida.idPerigo = perigo.id
''')

Note que a alface não aparece aqui. 

Se quisermos que todas as comidas apareçam neste join, mesmo que não estejam associadas a um perigo, como fazer? Para isso servem os ***outer joins***. Vamos experimentar com o **`LEFT OUTER JOIN`**:

In [ ]:
db('''
SELECT 
    comida.Nome, 
    perigo.Nome 
FROM
    comida
    LEFT OUTER JOIN perigo ON comida.idPerigo = perigo.id
''')

Veja que agora podemos observar que a alface não oferece perigo! E se quisermos fazer o contrário: queremos que todos os perigos apareçam, mesmo que não exista comida associada? Vamos usar o **`RIGHT OUTER JOIN`**:

In [ ]:
db('''
SELECT 
    comida.Nome,
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
''')

Agora os perigos dermatológico e mental aparecem, mas não existem comidas associadas a eles!

Os dois tipos de join são redundantes: basta inverter a ordem das partes para trocar de `RIGHT OUTER JOIN` para `LEFT OUTER JOIN`. Por exemplo:

In [ ]:
db('''
SELECT
    comida.Nome,
    perigo.Nome
FROM
    perigo
    LEFT OUTER JOIN comida ON comida.idPerigo = perigo.id
''')

#### Vamos praticar

Liste os perigos que não estão associados a nenhuma comida.

In [ ]:
db('''
SELECT
    comida.Nome,
    perigo.Nome
FROM
    perigo
    LEFT OUTER JOIN comida ON comida.idPerigo = perigo.id
WHERE
    comida.idPerigo IS NULL
''')

## Finalizando o trabalho

Por fim, vamos fechar nossa conexão com o banco de dados!

In [ ]:
connection.close()

Se você tentar fechar a conexão duas vezes, nada acontecerá nesta biblioteca. Outras bibliotecas podem lançar terá uma exceção: consulte a documentação da sua biblioteca.

In [ ]:
connection.close()

Por hoje é só! Pratique com os exercícios do seu livro texto.